<a href="https://colab.research.google.com/github/userMondo/CryptoSMA200Analyzer/blob/main/CryptoMA200Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basically, this program will find the closest and below SMA200

Step-by-Step to run this code:
1. Install libraries
2. run cell below final version
3. after done fetching data, Play around with it!!


### Installation necessary libraries

In [ ]:
!pip install requests pandas ccxt numpy tabulate tqdm # FIRSTLY, Run this one to install necessary libraries (Once)

#Final Version

main code right here

In [ ]:
# @title
import ccxt
import pandas as pd
import numpy as np
from tqdm import tqdm

# Initialize the OKX exchange

#--------------------#--------------------#-------DEFINE YOUR EXCHANGE HERE-------#--------------------#--------------------#

exchange = ccxt.okx() # ex: ccxt.binance() | .karken() | here's link https://pypi.org/project/ccxt/ where you could find you your capable exchange

#--------------------#--------------------#--------------------#--------------------#--------------------#--------------------#
# Fetch all markets
markets = exchange.load_markets()

# Filter for symbols ending with USDT, excluding stablecoins
usdt_symbols = [symbol for symbol in markets if symbol.endswith('/USDT') and symbol not in ['BUSD/USDT', 'USDT/USDT', 'TUSD/USDT', 'USDC/USDT', 'DAI/USDT']]

# Define a function to fetch OHLCV data and calculate SMA200

#--------------------#--------------------#---YOU CAN ADJUST TIMEFRAME RIGHT HERE---#--------------------#--------------------#

def fetch_ohlcv_and_ma(symbol, timeframe='1h'): # YOU CAN ADJUST TIMEFRAME RIGHT HERE

#--------------------#--------------------#--------------------#--------------------#--------------------#--------------------#

# Define a function to fetch OHLCV data and calculate the Moving Average (MA) with smoothing
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=209)  # Ensure enough data for smoothing
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

    # Calculate the initial MA with a length of 200
    df['ma200'] = df['close'].rolling(window=200).mean()

    # Calculate the smoothing of the MA with an SMA of length 9
    df['smoothing_ma'] = df['ma200'].rolling(window=9).mean()

    return df

# Define a function to check if the current price is close to the smoothed MA value
def is_price_close_to_smoothed_ma(df):
    if len(df) < 209:  # Ensure there are enough data points for the smoothing process
        return False
    current_price = df['close'].iloc[-1]
    current_smoothed_ma = df['smoothing_ma'].iloc[-1]
    if np.isnan(current_smoothed_ma):
        return False
    return current_price > current_smoothed_ma and abs(current_price - current_smoothed_ma) < 0.05 * current_smoothed_ma

# Initialize a list to store results
results = []

# Iterate through the symbols and check conditions, with a progress bar
for symbol in tqdm(usdt_symbols, desc='Fetching data', unit='symbol'):
    try:
        df = fetch_ohlcv_and_ma(symbol)
        if is_price_close_to_smoothed_ma(df):
            current_price = df['close'].iloc[-1]
            current_smoothed_ma = df['smoothing_ma'].iloc[-1]
            proximity = (abs(current_price - current_smoothed_ma) / current_smoothed_ma) * 100  # Calculate proximity in percentage
            results.append([symbol, current_price, current_smoothed_ma, proximity])
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Check if there are any results
if results:
    # Create a DataFrame from the results
    results_df = pd.DataFrame(results, columns=['Symbol', 'Current Price', 'Smoothed MA', 'Proximity (%)'])
    # Sort the DataFrame by proximity to the smoothed MA value
    results_df.sort_values(by='Proximity (%)', inplace=True)

    # Display the sorted results in table form
    results_df.reset_index(drop=True, inplace=True)
    display(results_df)  # Use display() in Colab to show the DataFrame
else:
    print("No symbols met the criteria.")

# Version 6

In [ ]:
# @title
import ccxt
import pandas as pd
from IPython.display import display

def get_crypto_data():
    exchange = ccxt.okx() # OKX exan
    markets = exchange.load_markets()

    results = []

    for symbol in markets:
        if symbol.endswith('USDT') and not symbol.startswith(('USDT', 'USDC', 'BUSD', 'DAI', 'PAX', 'TUSD')):
            try:
                ohlcv = exchange.fetch_ohlcv(symbol, '1h', limit=200)
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

                current_price = df['close'].iloc[-1]
                sma200 = df['close'].rolling(window=200).mean().iloc[-1]

                if current_price > sma200 * 0.95 and current_price < sma200 * 1.05:
                    results.append({
                        'Symbol': symbol,
                        'Current Price': current_price,
                        'SMA200': sma200,
                        'Difference (%)': (current_price - sma200) / sma200 * 100
                    })
            except Exception as e:
                print(f"Error processing {symbol}: {str(e)}")

    return pd.DataFrame(results)

def main():
    df = get_crypto_data()
    df_sorted = df.sort_values('Difference (%)', key=abs).head(50)

    # Format the numeric columns
    df_sorted['Current Price'] = df_sorted['Current Price'].map('{:.2f}'.format)
    df_sorted['SMA200'] = df_sorted['SMA200'].map('{:.2f}'.format)
    df_sorted['Difference (%)'] = df_sorted['Difference (%)'].map('{:.2f}'.format)

    # Reset the index and hide it
    df_sorted = df_sorted.reset_index(drop=True)

    # Display the table
    display(df_sorted.style
            .set_properties(**{'text-align': 'right'}, subset=['Current Price', 'SMA200', 'Difference (%)'])
            .set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'center')]},
                {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
                {'selector': '.index_name', 'props': [('display', 'none')]}
            ])
    )

if __name__ == "__main__":
    main()

ExchangeNotAvailable: binance GET https://api.binance.com/api/v3/exchangeInfo 451  {
  "code": 0,
  "msg": "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."
}

# Version 5

In [ ]:
import ccxt
import pandas as pd
from tabulate import tabulate

def get_crypto_data():
    # Use OKX exchange
    exchange = ccxt.okx()
    markets = exchange.load_markets()

    results = []

    for symbol in markets:
        if symbol.endswith('USDT') and not symbol.startswith(('USDT', 'USDC', 'BUSD', 'DAI', 'PAX', 'TUSD')):
            try:
                ohlcv = exchange.fetch_ohlcv(symbol, '1h', limit=200)
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

                current_price = df['close'].iloc[-1]
                sma200 = df['close'].rolling(window=200).mean().iloc[-1]

                if current_price > sma200 * 0.95 and current_price < sma200 * 1.05:
                    results.append({
                        'Symbol': symbol,
                        'Current Price': current_price,
                        'SMA200': sma200,
                        'Difference': (current_price - sma200) / sma200 * 100
                    })
            except Exception as e:
                print(f"Error processing {symbol}: {str(e)}")

    return pd.DataFrame(results)

def main():
    df = get_crypto_data()
    df_sorted = df.sort_values('Difference', key=abs)

    print(tabulate(df_sorted, headers='keys', tablefmt='pipe', floatfmt='.2f'))

if __name__ == "__main__":
    main()

|    | Symbol          |   Current Price |   SMA200 |   Difference |
|---:|:----------------|----------------:|---------:|-------------:|
| 43 | REVV/USDT       |            0.01 |     0.01 |        -0.01 |
| 39 | OMG/USDT        |            0.29 |     0.29 |         0.09 |
| 18 | ENS/USDT        |           26.48 |    26.53 |        -0.18 |
| 22 | ICE/USDT        |            0.00 |     0.00 |        -0.20 |
| 72 | ENS/USDT:USDT   |           26.44 |    26.53 |        -0.33 |
| 56 | WIFI/USDT       |            0.05 |     0.05 |         0.37 |
| 80 | TRX/USDT:USDT   |            0.13 |     0.14 |        -0.48 |
| 44 | SAFE/USDT       |            1.42 |     1.43 |        -0.48 |
| 54 | TRX/USDT        |            0.13 |     0.14 |        -0.49 |
| 35 | NFT/USDT        |            0.00 |     0.00 |         0.50 |
| 76 | NFT/USDT:USDT   |            0.00 |     0.00 |         0.56 |
| 55 | UNI/USDT        |            8.11 |     8.16 |        -0.60 |
| 81 | UNI/USDT:USDT   |          

#Version 4

In [ ]:
import ccxt
import pandas as pd
import numpy as np

# Initialize the exchange
exchange = ccxt.binance()

# Fetch all markets
markets = exchange.load_markets()

# Filter for symbols ending with USDT, excluding stablecoins
usdt_symbols = [symbol for symbol in markets if symbol.endswith('/USDT') and symbol not in ['BUSD/USDT', 'USDT/USDT', 'TUSD/USDT', 'USDC/USDT']]

# Define a function to fetch OHLCV data and calculate SMA200
def fetch_ohlcv_and_sma200(symbol, timeframe='1h'):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['sma200'] = df['close'].rolling(window=200).mean()
    return df

# Define a function to check if the current price is close to the SMA200 value
def is_price_close_to_sma200(df):
    current_price = df['close'].iloc[-1]
    current_sma200 = df['sma200'].iloc[-1]
    if np.isnan(current_sma200):
        return False
    return current_price > current_sma200 and abs(current_price - current_sma200) < 0.05 * current_sma200

# Initialize a list to store results
results = []

# Iterate through the symbols and check conditions
for symbol in usdt_symbols:
    try:
        df = fetch_ohlcv_and_sma200(symbol)
        if is_price_close_to_sma200(df):
            current_price = df['close'].iloc[-1]
            current_sma200 = df['sma200'].iloc[-1]
            proximity = abs(current_price - current_sma200) / current_sma200
            results.append((symbol, current_price, current_sma200, proximity))
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Sort the results by proximity to the SMA200 value
results.sort(key=lambda x: x[3])

# Print the sorted results
for result in results:
    print(f"Symbol: {result[0]}, Current Price: {result[1]}, SMA200: {result[2]}, Proximity: {result[3]}")




Symbol: PAX/USDT, Current Price: 0.9996, SMA200: 0.9995660000000001, Proximity: 3.401476240686307e-05
Symbol: FDUSD/USDT, Current Price: 1.0003, SMA200: 0.9998140000000001, Proximity: 0.0004860904128166593
Symbol: GTO/USDT, Current Price: 0.01233, SMA200: 0.012313750000000002, Proximity: 0.0013196629783777405
Symbol: AEUR/USDT, Current Price: 1.0883, SMA200: 1.086074, Proximity: 0.002049584098321165
Symbol: EUR/USDT, Current Price: 1.0884, SMA200: 1.0861225, Proximity: 0.002096908958243598
Symbol: DAI/USDT, Current Price: 1.0189, SMA200: 1.0155585, Proximity: 0.00329030774691944
Symbol: USDSB/USDT, Current Price: 0.9997, SMA200: 0.9957645, Proximity: 0.00395223971129718
Symbol: MLN/USDT, Current Price: 20.7, SMA200: 20.547400000000003, Proximity: 0.007426730389246136
Symbol: BCHABC/USDT, Current Price: 220.08, SMA200: 218.12555, Proximity: 0.008960206633289903
Symbol: MFT/USDT, Current Price: 0.005254, SMA200: 0.005202945, Proximity: 0.009812711839160394
Symbol: SFP/USDT, Current Price

#VERSION $3

In [ ]:
import ccxt
import pandas as pd

# Initialize the exchange
exchange = ccxt.binance()

# Get all symbols traded on the exchange
markets = exchange.load_markets()

# Function to fetch OHLCV data and calculate SMA200
def fetch_sma200(symbol, timeframe='1h', limit=500):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['SMA200'] = df['close'].rolling(window=200).mean()
    return df

# Filter out stablecoins
symbols = [symbol for symbol in markets if not symbol.endswith('/USDT') and not symbol.endswith('/BUSD') and not symbol.endswith('/USDC')]

# Fetch SMA200 for each symbol and calculate the distance from the current price
results = []
for symbol in symbols:
    try:
        df = fetch_sma200(symbol)
        if len(df) < 200:
            continue  # Not enough data to calculate SMA200
        current_close = df.iloc[-1]['close']
        sma200 = df.iloc[-1]['SMA200']
        distance = abs(current_close - sma200)
        results.append((symbol, current_close, sma200, distance))
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Sort results by distance and filter the closest 100
results_sorted = sorted(results, key=lambda x: x[3])[:100]

# Print results
for result in results_sorted:
    print(f"Symbol: {result[0]}, Current Price: {result[1]}, SMA200: {result[2]}, Distance: {result[3]}")


Symbol: WAVES/USDT:USDT, Current Price: 1.3355, SMA200: 1.3355, Distance: 0.0
Symbol: OCEAN/USDT:USDT, Current Price: 0.707, SMA200: 0.707, Distance: 0.0
Symbol: CVC/USDT:USDT, Current Price: 0.10201, SMA200: 0.10201, Distance: 0.0
Symbol: CTK/USDT:USDT, Current Price: 1.0106, SMA200: 1.0106, Distance: 0.0
Symbol: BTCST/USDT:USDT, Current Price: 319.408, SMA200: 319.408, Distance: 0.0
Symbol: SC/USDT:USDT, Current Price: 0.003653, SMA200: 0.003653, Distance: 0.0
Symbol: DGB/USDT:USDT, Current Price: 0.01922, SMA200: 0.01922, Distance: 0.0
Symbol: RAY/USDT:USDT, Current Price: 0.248, SMA200: 0.248, Distance: 0.0
Symbol: FTT/USDT:USDT, Current Price: 1.59, SMA200: 1.59, Distance: 0.0
Symbol: CVX/USDT:USDT, Current Price: 2.374, SMA200: 2.374, Distance: 0.0
Symbol: AGIX/USDT:USDT, Current Price: 0.7009, SMA200: 0.7009, Distance: 0.0
Symbol: IDEX/USDT:USDT, Current Price: 0.0561, SMA200: 0.0561, Distance: 0.0
Symbol: RAD/USDT:USDT, Current Price: 1.6874, SMA200: 1.6874, Distance: 0.0
Symbo

# Version 3

In [ ]:
# prompt: Please Provide a code for fetching cryptocurrencies' historical data and detecting every Symbol by its closest price with SMA200 in Python. using CCXT API. The app can sort the closest values in the 100 range. in timeframe 1hour. Exclude stable coins

import requests
import pandas as pd
import ccxt
import numpy as np

# Initialize the exchange
exchange = ccxt.binance()

# Define the timeframe and limit
timeframe = '1h'
limit = 1000  # Adjust as needed

# Get the list of available symbols
markets = exchange.load_markets()
symbols = list(markets.keys())

# Exclude stablecoins (adjust the list as needed)
stablecoins = ['USDT', 'USDC', 'BUSD', 'DAI', 'TUSD', 'PAX', 'USDP']
symbols = [s for s in symbols if not any(stable in s for stable in stablecoins)]

# Fetch historical data and calculate SMA200
results = []
for symbol in symbols:
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df['SMA200'] = df['close'].rolling(200).mean()

        # Calculate the distance to SMA200 for the last closing price
        last_price = df['close'].iloc[-1]
        sma200 = df['SMA200'].iloc[-1]
        distance = abs(last_price - sma200)

        results.append({'symbol': symbol, 'distance': distance})

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Create a DataFrame and sort by distance
df_results = pd.DataFrame(results)
df_results_sorted = df_results.sort_values('distance')

# Filter for distances within the 100 range
df_filtered = df_results_sorted[df_results_sorted['distance'] <= 100]

# Print the results
print(df_filtered)


                    symbol  distance
1571  GALA/USD:GALA-221226    0.0000
1577  ROSE/USD:ROSE-240521    0.0000
1579    ICX/USD:ICX-240520    0.0000
1578    XTZ/USD:XTZ-240520    0.0000
1583    VET/USD:VET-221226    0.0000
...                    ...       ...
824             MATIC/BIDR   83.3050
1174               ETC/TRY   85.0990
1092               INJ/TRY   87.4725
999                NEO/RUB   88.4350
842                SOL/BRL   91.9330

[1513 rows x 2 columns]


# VERSION 2

In [ ]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def fetch_ohlcv(exchange, symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

def calculate_sma(df, window):
    return df['close'].rolling(window=window).mean()

def main():
    exchange = ccxt.binance()
    timeframe = '1h'
    limit = 1000  # Fetch 1000 candles to ensure we have enough data for SMA200

    # Fetch all available symbols
    markets = exchange.load_markets()
    symbols = [symbol for symbol in markets.keys() if not any(stablecoin in symbol for stablecoin in ['USDT', 'USDC', 'BUSD', 'DAI', 'TUSD'])]

    results = []

    for symbol in symbols:
        try:
            df = fetch_ohlcv(exchange, symbol, timeframe, limit)
            if len(df) < 200:
                continue

            df['sma200'] = calculate_sma(df, 200)
            last_close = df['close'].iloc[-1]
            last_sma200 = df['sma200'].iloc[-1]

            if not np.isnan(last_sma200):
                difference = abs(last_close - last_sma200)
                percentage_difference = (difference / last_sma200) * 100
                results.append({
                    'symbol': symbol,
                    'close': last_close,
                    'sma200': last_sma200,
                    'difference': difference,
                    'percentage_difference': percentage_difference
                })

        except Exception as e:
            print(f"Error processing {symbol}: {str(e)}")

    # Sort results by percentage difference
    sorted_results = sorted(results, key=lambda x: x['percentage_difference'])

    # Display top 100 closest to SMA200
    print("Top 100 cryptocurrencies closest to their SMA200:")
    for i, result in enumerate(sorted_results[:100], 1):
        print(f"{i}. {result['symbol']}: Close: {result['close']:.8f}, SMA200: {result['sma200']:.8f}, Difference: {result['difference']:.8f} ({result['percentage_difference']:.2f}%)")

if __name__ == "__main__":
    main()

Top 100 cryptocurrencies closest to their SMA200:
1. GALA/USD:GALA-221226: Close: 0.01833000, SMA200: 0.01833000, Difference: 0.00000000 (0.00%)
2. ROSE/USD:ROSE-240521: Close: 0.09505000, SMA200: 0.09505000, Difference: 0.00000000 (0.00%)
3. XTZ/USD:XTZ-240520: Close: 0.91890000, SMA200: 0.91890000, Difference: 0.00000000 (0.00%)
4. ICX/USD:ICX-240520: Close: 0.22020000, SMA200: 0.22020000, Difference: 0.00000000 (0.00%)
5. APE/USD:APE-240521: Close: 1.29800000, SMA200: 1.29800000, Difference: 0.00000000 (0.00%)
6. VET/USD:VET-221226: Close: 0.01614000, SMA200: 0.01614000, Difference: 0.00000000 (0.00%)
7. ZIL/USD:ZIL-221226: Close: 0.01900000, SMA200: 0.01900000, Difference: 0.00000000 (0.00%)
8. KNC/USD:KNC-240520: Close: 0.58090000, SMA200: 0.58090000, Difference: 0.00000000 (0.00%)
9. USDSB/USDS: Close: 0.99990000, SMA200: 0.99991350, Difference: 0.00001350 (0.00%)
10. BLUR/BTC: Close: 0.00000290, SMA200: 0.00000290, Difference: 0.00000000 (0.00%)
11. WBETH/ETH: Close: 1.04340000,

# VERSION 1

In [ ]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta
import time

def fetch_ohlcv(exchange, symbol, timeframe, limit):
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        return df
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def calculate_ma200(df):
    return df['close'].rolling(window=200).mean().iloc[-1]

def main():
    exchange = ccxt.binance()
    timeframe = '1h'
    limit = 1000  # Fetch 1000 candles to ensure we have enough data for MA200

    # Get all available symbols
    markets = exchange.load_markets()
    symbols = [symbol for symbol in markets.keys() if symbol.endswith('/USDT')]

    results = []

    for symbol in symbols:
        print(f"Processing {symbol}...")
        df = fetch_ohlcv(exchange, symbol, timeframe, limit)

        if df is not None and len(df) >= 200:
            current_price = df['close'].iloc[-1]
            ma200 = calculate_ma200(df)
            difference = abs(current_price - ma200)
            percentage_difference = (difference / ma200) * 100

            results.append({
                'symbol': symbol,
                'current_price': current_price,
                'ma200': ma200,
                'difference': difference,
                'percentage_difference': percentage_difference
            })

        time.sleep(0.1)  # To avoid hitting rate limits

    # Sort results by percentage difference
    sorted_results = sorted(results, key=lambda x: x['percentage_difference'])

    # Display top 100 closest to MA200
    print("\nTop 100 cryptocurrencies closest to their MA200:")
    for i, result in enumerate(sorted_results[:100], 1):
        print(f"{i}. {result['symbol']}: Current Price: {result['current_price']:.8f}, MA200: {result['ma200']:.8f}, Difference: {result['percentage_difference']:.2f}%")

if __name__ == "__main__":
    main()

Processing BTC/USDT...
Processing ETH/USDT...
Processing BNB/USDT...
Processing BCC/USDT...
Processing NEO/USDT...
Processing LTC/USDT...
Processing QTUM/USDT...
Processing ADA/USDT...
Processing XRP/USDT...
Processing EOS/USDT...
Processing TUSD/USDT...
Processing IOTA/USDT...
Processing XLM/USDT...
Processing ONT/USDT...
Processing TRX/USDT...
Processing ETC/USDT...
Processing ICX/USDT...
Processing VEN/USDT...
Processing NULS/USDT...
Processing VET/USDT...
Processing PAX/USDT...
Processing BCHABC/USDT...
Processing BSV/USDT...
Processing USDC/USDT...
Processing LINK/USDT...
Processing WAVES/USDT...
Processing BTT/USDT...
Processing USDS/USDT...
Processing ONG/USDT...
Processing HOT/USDT...
Processing ZIL/USDT...
Processing ZRX/USDT...
Processing FET/USDT...
Processing BAT/USDT...
Processing XMR/USDT...
Processing ZEC/USDT...
Processing IOST/USDT...
Processing CELR/USDT...
Processing DASH/USDT...
Processing NANO/USDT...
Processing OMG/USDT...
Processing THETA/USDT...
Processing ENJ/U

# Support me:
- USDT(tron): TEgBzL1EQ1rZKwmMBu9ZFuTJ5zTpxvmqxE
- BNB: bnb1u68drmfd4teaqyag3kpa72dgcx3sxf22ymdejv
- ETH(erc20): 0x488e8DB935A3555fE06c087Ea003DF31555f80cD
- BTC: bc1qs0yvpc2l4wrk4u2vh49pwv7ays4rfcll73nzq5


[Membership!!](https://buymeacoffee.com/mond)